In [8]:
import os
import h5py
import numpy as np
from functions.preprocess import input_shaping, split_index
from functions.decoders import CNNDecoder
from functions.metrics import compute_rmse, compute_pearson
from functions.channel_mapping import channel_mapping
from tensorflow.keras import initializers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers.recurrent import LSTM
from tensorflow.keras.layers import Dense, Activation, Lambda , Input , Flatten ,Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from keras.utils import np_utils
from tensorflow import random
import time as timer
from keras.models import load_model
from functions.metrics import compute_rmse, compute_pearson

# from torch.autograd import Variable
# import torch
# from torchinfo import summary
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)


In [21]:

seed = 2020 # random seed for reproducibility

print ("Starting simulation")
run_start = timer.time()

feature_list = ['sua_rate', 'mua_rate']
feature = feature_list[1] # select which spike feature: SUA=0, MUA=1

# specify filename to be processed (choose from the list available at https://zenodo.org/record/583331)
file_name = 'indy_20160915_01'          # file name
kinematic_folder = 'kinematic_data/'    # kinematic data folder
feature_folder = 'spike_data/features/' # spike features folder
result_folder = 'results/'              # results folder
wdw_time = 0.128 # window size in second
lag = -32 # lag between kinematic and feature data (minus indicate feature lagging behaind kinematic)
delta_time = 0.004 # sampling interval in second
wdw_samp = int(round(wdw_time/delta_time))
ol_samp = wdw_samp-1

# open spike features from hdf5 file
feature_file = feature_folder+file_name+'_spike_features_'+str(int(wdw_time*1e3))+'ms.h5'
print ("Loading input features from file: "+feature_file)
with h5py.File(feature_file,'r') as f:
    input_feature = f[feature][()]
channel_mapping_file ='raw_data/indy_20170127_03.nwb' #r'F:/dropbox/Dropbox (Imperial NGNI)/NGNI Share/Workspace/Zheng/Research_Topics/signal processing plantform/prediction/decoding/raw_data/indy_20170127_03.nwb'

with h5py.File(channel_mapping_file, "r") as f:
    channel_loc = f['/general/extracellular_ephys/electrode_map'][()]
input_feature = channel_mapping(input_feature,channel_loc)
# open kinematic data from hdf5 file
kinematic_file = kinematic_folder+file_name+'_kinematic_data.h5'
print ("Loading kinematic data from file: "+kinematic_file)
with h5py.File(kinematic_file,'r') as f:
    cursor_vel = f['cursor_vel'][()] # in mm/s

# set QRNN hyperparameters
units = 300 # SUA: 200, MUA: 150
window_size = 2
epochs = 15
batch_size = 32
dropout = 0.
lrate = 0.003

num_fold = 5 # number of folds
 # SUA: 0.002, MUA: 0.0035 
momentum = 0.9
weight_decay = 1e-4
print("Hyperparameters >> units={}, window_size={}, epochs={}, batch_size={}, dropout={:.1f}, lrate={:.4f}".format(
    units, window_size, epochs, batch_size, dropout, lrate))          

# Define dictionary of parameters    
num_layers = 1 # number of layers
optimizer = 'adam' # optimizer
timesteps = 1 # number of timesteps (lag + current)
kernel_init = initializers.glorot_uniform(seed=seed) 
recur_init =  initializers.Orthogonal(seed=seed)
input_dim = timesteps#input_feature.shape[1] # input dimension
output_dim = cursor_vel.shape[1] # output dimension
verbose = 0

setting1 = 2
setting2 =3
# in_planes = {1:4,2:8,3:16,4:32}

featureses = {1:[4,4,8,16],2:[8,8,16,32],3:[16,16,32,64],4:[32,32,64,128]} 
num_blockses = {1:[1],2:[1,1],3:[1,1,1],4:[2],5:[2,2],6:[2,2,2],7:[3],8:[3,3],9:[3,3,3]}
in_plane = [16]
features_out = 64
stride = [1,1,2,2]

features = [16,16,32,64]
num_blocks = [3,3,3]


Starting simulation
Loading input features from file: spike_data/features/indy_20160915_01_spike_features_128ms.h5
Loading kinematic data from file: kinematic_data/indy_20160915_01_kinematic_data.h5
Hyperparameters >> units=300, window_size=2, epochs=15, batch_size=32, dropout=0.0, lrate=0.0030


In [23]:
# initialise performance scores (RMSE and CC) with nan values
loss_train = np.full((num_fold, epochs), np.nan)
loss_valid = np.copy(loss_train)
rmse_valid = np.full((num_fold, output_dim), np.nan)
rmse_test = np.copy(rmse_valid)
cc_valid = np.copy(rmse_valid)
cc_test = np.copy(rmse_valid)
time_train = np.full((num_fold), np.nan)
time_test = np.copy(time_train) 


print ("Formatting input feature data")
tstep = timesteps # timestep (lag + current) samples
stride = 1 # number of samples to be skipped
X_in = input_shaping(input_feature.reshape(input_feature.shape[0],-1), timesteps, 1)
X_in = X_in.reshape(X_in.shape[0],timesteps,10,10)

print ("Formatting output (kinematic) data")
diff_samp = cursor_vel.shape[0]-X_in.shape[0]
Y_out = cursor_vel[diff_samp:,:] # in mm/s (remove it for new corrected velocity)

print ("Splitting input dataset into training, validation, and testing subdataset")
all_train_idx, all_valid_idx, all_test_idx = split_index(Y_out, num_fold)
global temp
for i in range(num_fold): 
    train_idx = all_train_idx[i]
    valid_idx = all_valid_idx[i]
    test_idx = all_test_idx[i]
    
    # specify training dataset
    X_train = X_in[train_idx,:]            
    Y_train = Y_out[train_idx,:]
    
    # specify validation dataset
    X_valid = X_in[valid_idx,:]
    Y_valid = Y_out[valid_idx,:]
    
    # specify validation dataset
    X_test = X_in[test_idx,:]
    Y_test = Y_out[test_idx,:]
    
    epsilon = 1e-4
    # Standardize (z-score) input dataset
    X_train_mean = np.nanmean(X_train, axis=0)
    X_train_std = np.nanstd(X_train, axis=0) 
    X_train = (X_train - X_train_mean)/(X_train_std+epsilon)
    X_valid = (X_valid - X_train_mean)/(X_train_std +epsilon)
    X_test = (X_test - X_train_mean)/(X_train_std +epsilon)
    
    # Zero mean (centering) output dataset
    Y_train_mean = np.nanmean(Y_train, axis=0)
    Y_train_std = np.nanstd(Y_train, axis=0) 
    Y_train = (Y_train - Y_train_mean)/(Y_train_std+epsilon)
    Y_valid = (Y_valid - Y_train_mean)/(Y_train_std +epsilon)
    Y_test = (Y_test - Y_train_mean)/(Y_train_std +epsilon)
    # Y_train = Y_train - Y_train_mean 
    # Y_valid = Y_valid - Y_train_mean
    # Y_test = Y_test - Y_train_mean
           
    #Re-align data to take lag into account
    if lag < 0:
        X_train = X_train[:lag,:] # remove lag first from end (X lag behind Y)
        Y_train = Y_train[-lag:,:] # reomve lag first from beginning
        X_valid = X_valid[:lag,:]
        Y_valid = Y_valid[-lag:,:]
        X_test = X_test[:lag,:]
        Y_test = Y_test[-lag:,:]
    if lag > 0:
        X_train = X_train[lag:,:] # reomve lag first from beginning
        Y_train = Y_train[:-lag,:] # remove lag first from end (X lead in front of Y)
        X_valid = X_valid[lag:,:]
        Y_valid = Y_valid[:-lag,:]            
        X_test = X_test[lag:,:]
        Y_test = Y_test[:-lag,:]
        
    # set seed to get reproducible results
np.random.seed(seed)
random.set_seed(seed)
X_train = np.moveaxis(X_train,1,-1)
X_valid = np.moveaxis(X_valid,1,-1)
X_test = np.moveaxis(X_test,1,-1)
print(X_train.shape)
print(Y_train.shape)

model_resnet = tf.keras.applications.ResNet50(include_top = False,input_shape=(32,32,1), weights=None)

output = model_resnet.output
output = Flatten()(output)
output = Dense(units=2)(output)
model = Model(model_resnet.input, output)
print(model.input)
print(model.output)
inp = Input(shape=(10, 10, 1))
out = Lambda(lambda image: tf.compat.v1.image.resize_images(image, (32, 32)))(inp)
out = model(out)
model = Model(inp, out)
print(model.output)
print(model.input)
model.summary()

opt = Adam(lr=1e-3, decay=1e-3 / 200)
model.compile(loss="mean_absolute_percentage_error", optimizer=opt)

model.fit(X_train, Y_train, validation_data=(X_valid, Y_valid), epochs=15, batch_size=32,verbose=True)
model.summary()

Formatting input feature data
Formatting output (kinematic) data
Splitting input dataset into training, validation, and testing subdataset
(57124, 10, 10, 1)
(57124, 2)
KerasTensor(type_spec=TensorSpec(shape=(None, 32, 32, 1), dtype=tf.float32, name='input_7'), name='input_7', description="created by layer 'input_7'")
KerasTensor(type_spec=TensorSpec(shape=(None, 2), dtype=tf.float32, name=None), name='dense_36/BiasAdd:0', description="created by layer 'dense_36'")
KerasTensor(type_spec=TensorSpec(shape=(None, 2), dtype=tf.float32, name=None), name='model_6/dense_36/BiasAdd:0', description="created by layer 'model_6'")
KerasTensor(type_spec=TensorSpec(shape=(None, 10, 10, 1), dtype=tf.float32, name='input_8'), name='input_8', description="created by layer 'input_8'")
Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 10, 10, 1)]       0         
       

In [24]:

def representative_dataset():
    for _ in range(100):
      data = np.random.rand(1, 10, 10, 1)
      yield [data.astype(np.float32)]


model.save('my_model.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
# converter.optimizations = [tf.lite.Optimize.DEFAULT]
# converter.representative_dataset = representative_dataset
# # Ensure that if any ops can't be quantized, the converter throws an error
# converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# # Set the input and output tensors to uint8 (APIs added in r2.3)
# converter.inference_input_type = tf.uint8
# converter.inference_output_type = tf.uint8
tflite_model = converter.convert()
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)